# Brief Introduction to Compiling and Debugging CUDA in Colab

This is a short summary of how to compile and debug CUDA programs in the Google Colab environment. Many thanks to Elisabeth Brunet for some of the example code.


### Compiling
1. Create a file with the program code. To do so, put your pogram into a code cell where the first line is<br>
`%%writefile prog.cu` <br>
Executing this cell with write its content to the file `prog.cu` (the name is arbitrary).
2. Compile your file by calling the compiler `nvcc` from the shell:<br>
```!nvcc -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver  -arch=sm_35 -Wno-deprecated-gpu-targets prog.cu```
3. Run your program:<br>
`!./a.out`

Here's a quick example. First, the program:

In [ ]:
%%writefile prog.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}
int main() {
  int res=0;
  int *d_res;
  // reserve memory for the result on the GPU
  cudaMalloc((void**)&d_res, sizeof(int));
  // Launch add() kernel on GPU, 
  // which writes its result to address d_res on GPU
  add<<<1,1>>>(2, 2, d_res);
  // wait for the GPU to finish
  cudaDeviceSynchronize();
  // copy result back to CPU
  cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost);
  // print result
  printf("2 + 2 = %d\n", res);
  return EXIT_SUCCESS;
}

Overwriting prog.cu


Let's compile the program. We need to call `nvcc` with a shell command. In Jupyter, shell commands start with `!`. We need to include the directories for the Cuda include files (`-I`)  and Cuda libraries (`-L`). Let's link also the cuBlas and cuSolver libraries, since you might need them at some point. We don't specify the name of the executable, so it will be the default `a.out`.  If everything goes well, executing the cell does not give any output. If the compiler has problems, you will see the error messages:

In [ ]:
!nvcc -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver -arch=sm_35 -Wno-deprecated-gpu-targets prog.cu

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


Let's run the program. Again, `!` indicates a shell command, and we must prefix the name of the executable with `./` so the shell looks for it in the current directory:

In [ ]:
!./a.out

2 + 2 = 4


## Debugging
Here are some tips on debugging if your program crashes:
1. The interface between the Jupyter Notebook and the executed program is a little fragile. So if your program crashes, there might not be ANY output at all, even if you have `printf` everywhere.
2. If you do use `printf`, be sure to flush the buffer by adding a line break at the end. This applies to any C program. Example:<br> `printf("Works up to here\n);`
2. Be sure to add error checks to EVERY cuda call (including cudaMalloc, cudaMemcpy, etc.) and call `cudaPeekAtLastError()` after kernel calls.
2. A frequent mistake is to forget that the CPU doesn't wait for kernel calls to finish. To wait for the GPU after a kernel call, use `cudaDeviceSynchronize()`.
If your program still crashes without output, the last resort is calling the debugger. 

To debug with `cuda-gdb`, you need to compile as described above, adding the options "-g -G" so that debugging symbols are included:

In [ ]:
!nvcc -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver prog.cu

The debugger is interactive (you are expected to type commands as you go along), but running programs in Jupyter Notebooks is not. So you need to write your commands to a file.
Typical commands would go like this:
1. set the debugger up to check lots of possible errors:
  1. memory checks `memcheck on`,
  2. stop in case of API failures `api_failures stop`,
  3. stop on exceptions `catch throw`, 
2. run the program (possibly with command line options) `r option1 option2`
, 
3. show the kernel call stack (GPU) `bt`, 
4. print all local variables `info locals`, 
5. switch to the host thread `thread 1` and show the host program call stack (CPU) `bt`.

You can use `%%writefile` to create a file `tmp.txt` with commands:


In [ ]:
%%writefile tmp.txt
set cuda memcheck on
set cuda api_failures stop
catch throw
r
bt
info locals
thread 1
bt


Overwriting tmp.txt


For a more compact solution, here's a one-liner shell command to write  commands to the file `tmp.txt`:

In [ ]:
!printf "set cuda memcheck on\nset cuda api_failures stop\ncatch throw\nr\nbt\ninfo locals\nthread 1\nbt\n" > tmp.txt

Now call the debugger with your program and execute the commands from tmp.txt. If your program terminates fine, `cuda-gdb` will complain that there's no stack (since the program finished):

In [ ]:
! cuda-gdb -batch -x tmp.txt ./a.out

Catchpoint 1 (throw)
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
[Detaching after fork from child process 505]
[New Thread 0x7f33d491e700 (LWP 509)]
[New Thread 0x7f33d411d700 (LWP 510)]
Cuda API error detected: cudaLaunchKernel returned (0xd1)
#0  0x00007f33d4d0e3e0 in cudbgReportDriverApiError () from /usr/lib64-nvidia/libcuda.so.1
#1  0x00007f33d4d17aa1 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#2  0x00007f33d4c6b1f6 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#3  0x00007f33d4c99373 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#4  0x00005627ac169475 in cudaLaunchKernel ()
#5  0x00005627ac11f4b7 in cudaLaunchKernel<char> (func=0x5627ac11f38e <add(int, int, int*)> "UH\211\345H\203\354\020\211}\374\211u\370H\211U\360H\213U\360\213M\370\213E\374\211Ή\307\350\206\376\377\377\220\311\303UH\211\345H\203\354\020H\211}\370H\213E\370H\211\005\023\035(", gridDim=..., blockDim=..., args=0x7ffe4824a580, sharedMem=0,

So let's look at a program with an error:

In [ ]:
%%writefile prog.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}
int main() {
  int res=0;
  int *d_res;
  // suppose we forgot malloc: cudaMalloc((void**)&d_res, sizeof(int));
  add<<<1,1>>>(2, 2, d_res);
  cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost);
  printf("2 + 2 = %d\n", res);

  return EXIT_SUCCESS;
}

Overwriting prog.cu


Compiling this faulty code gives a warning, but the program compiles and runs fine. It just doesn't give the correct result:

In [ ]:
!nvcc -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver prog.cu
!./a.out

prog.cu(11): warning: variable "d_res" is used before its value is set

prog.cu(11): warning: variable "d_res" is used before its value is set

2 + 2 = 0


We can try to catch this problem in two ways: use a debugger to check for memory errors and other problems, or add explicit error checks (which is discussed in the  next section).

Let's compile for debugging, write the debug commands to `tmp.txt` and call `cuda-gdb`:

In [ ]:
!nvcc -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver prog.cu
!printf "set cuda memcheck on\nset cuda api_failures stop\ncatch throw\nr\nbt\ninfo locals\nthread 1\nbt\n" > tmp.txt
!cuda-gdb -batch -x tmp.txt ./a.out

prog.cu(11): warning: variable "d_res" is used before its value is set

prog.cu(11): warning: variable "d_res" is used before its value is set

Catchpoint 1 (throw)
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
[Detaching after fork from child process 588]
[New Thread 0x7f933fe3e700 (LWP 592)]
[New Thread 0x7f933f63d700 (LWP 593)]
Cuda API error detected: cudaLaunchKernel returned (0xd1)
#0  0x00007f934022e3e0 in cudbgReportDriverApiError () from /usr/lib64-nvidia/libcuda.so.1
#1  0x00007f9340237aa1 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#2  0x00007f934018b1f6 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#3  0x00007f93401b9373 in ?? () from /usr/lib64-nvidia/libcuda.so.1
#4  0x0000564f8566b465 in cudaLaunchKernel ()
#5  0x0000564f856214a1 in cudaLaunchKernel<char> (func=0x564f85621378 <add(int, int, int*)> "UH\211\345H\203\354\020\211}\374\211u\370H\211U\360H\213U\360\213M\370\213E\374\211Ή\307\350\206\376

We get an exception and lots of information. There is an illegal address detected in line 5 of `prog.cu`, which is in kernel `add`. We also see the call stack for the host, which shows that the kernel is called in `main()` at line 11. To see line numbers in Colab, use Ctrl + M + L and take into account that the `%%writefile` increases the line number by one.

## Error Checking in Cuda
The CPU is not notified when an error occurs on the GPU. This means that you need to check after every Cuda call whether there was an error, by looking at the return value of the Cuda function. Since kernel calls don't have a return value, you need to call `cudaPeekAtLastError()`. The following code from Stack Overflow works nicely. It uses a macro to show the function name and line number if there is an error:

```
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
```

Our faulty example program looks as follows with error checking. Note the `gpuErrchk` on every line involving Cuda and the `cudaPeekAtLastError()` after the kernel call:

In [ ]:
%%writefile prog.cu
#include <stdio.h>
#include <stdlib.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}
int main() {
  int res=0;
  int *d_res;
  // suppose we forgot this:  gpuErrchk( cudaMalloc((void**)&d_res, sizeof(int)) );
  add<<<1,1>>>(2, 2, d_res);
  gpuErrchk( cudaPeekAtLastError() );
  gpuErrchk( cudaDeviceSynchronize() );
  gpuErrchk( cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost) );
  // print result
  printf("2 + 2 = %d\n", res);
  return EXIT_SUCCESS;
}

Overwriting prog.cu


Let's see if it still compiles and runs:

In [ ]:
!nvcc -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver prog.cu

prog.cu(20): warning: variable "d_res" is used before its value is set

prog.cu(20): warning: variable "d_res" is used before its value is set



In [ ]:
!./a.out

GPUassert: no kernel image is available for execution on the device prog.cu 21


Thanks to the error checking, we get an error message. That's much better than the program finishing its computation but giving a wrong result.
Note that here, we only know that the last kernel caused the error, but not where inside the kernel the error occured. You can use the debugger to find out more details, as described above.